# Electric vehicle charging station locations - Worldwide 
This notebook has been used to extract API data from Open Charge Map.

API used to extract all recorded charging stations in the world

https://openchargemap.org/site

API documentation: 
https://openchargemap.org/site/develop/api

## Country Codes
Country codes have been retreived from :
https://datahub.io/core/country-list/r/data.json

In [1]:
# Python Dependencies
import requests
import json
import pprint
import pymongo
import time
from config import api_key # Open Charge Map API

In [2]:
# Get a small record set from the API to better understand the JSON structure 

units = "metric"
country_code = "CN"
count = 1
distanceunit = "KM"
url = f"https://api.openchargemap.io/v3/poi/?key={api_key}&output=json&countrycode={country_code}&distanceunit={distanceunit}&maxresults={count}&compact=true&verbose=false"
# url = "https://api.openchargemap.io/v3/referencedata/"

response = requests.get(url).json()

pprint.pprint(response)

[{'AddressInfo': {'AddressLine1': '7388 Humin Road',
                  'CountryID': 50,
                  'DistanceUnit': 0,
                  'ID': 164114,
                  'Latitude': 31.13140735,
                  'Longitude': 121.40009915,
                  'Title': 'Shanghai - Southern Bailian Supercharger',
                  'Town': 'Shanghai'},
  'Connections': [{'ConnectionTypeID': 27,
                   'CurrentTypeID': 30,
                   'ID': 235354,
                   'LevelID': 3,
                   'PowerKW': 120.0,
                   'Quantity': 10,
                   'StatusTypeID': 50}],
  'DataProviderID': 1,
  'DataQualityLevel': 1,
  'DateCreated': '2020-08-24T16:26:00Z',
  'DateLastStatusUpdate': '2020-08-24T16:27:00Z',
  'GeneralComments': 'Charging10 Superchargers, available 24/7, up to 120kW',
  'ID': 163757,
  'IsRecentlyVerified': False,
  'NumberOfPoints': 10,
  'OperatorID': 23,
  'OperatorsReference': '49613',
  'StatusTypeID': 50,
  'SubmissionStatusT

In [3]:
for location in response:
    print(f"{location['AddressInfo']['Town']} {location['AddressInfo']['Title']} {location['AddressInfo']['Latitude']} {location['AddressInfo']['Longitude']} {location['NumberOfPoints']} {location['GeneralComments']}")
    connections = location['Connections']
    for connection in connections:
        print(f"{connection['CurrentTypeID']} {connection['PowerKW']} {connection['LevelID']} {connection['Quantity']}")

Shanghai Shanghai - Southern Bailian Supercharger 31.13140735 121.40009915 10 Charging10 Superchargers, available 24/7, up to 120kW
30 120.0 3 10


## Country codes
Obtained from datahub.io and stroed in Mongo

In [4]:
# Obtain a list of Country Codes
url = "https://datahub.io/core/country-list/r/data.json"

countries = requests.get(url).json()

In [ ]:
pprint.pprint(countries)

In [5]:
# Connect to Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Use the 'electric_vehicles' database in Mongo
db = client.electric_vehicles

# Store in Mongo
db.country_codes.insert_many(countries)

## Charging Station Locations - Per Country
Get the data for each country and store in the Mongo Database

In [ ]:
# For testing only - loop through two countries only
countries = [{'Code': 'AU', 'Name': 'Australia'},
 {'Code': 'AT', 'Name': 'Austria'}]
for country in countries:  
    pprint.pprint(country)


In [6]:
# Connect to Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Use the 'electric_vehicles' database in Mongo
db = client.electric_vehicles

# Set up the Open Charge Map API parameters
units = "metric"
distanceunit = "KM"

# Loop through each country and use the country code for API call
for country in countries: 
    pprint.pprint(f"Trying {country['Name']} ....")
    url = f"https://api.openchargemap.io/v3/poi/?key={api_key}&output=json&countrycode={country['Code']}&distanceunit={distanceunit}"
    country_response = requests.get(url).json()
    
    for location in country_response:
        connections = []
        location_connections = location['Connections']
        for connection in location_connections:
            connection_dict = {'currentTypeID':connection['CurrentTypeID'], 'powerKW':connection['PowerKW'], 'levelID':connection['LevelID'], 'quantity': connection['Quantity']}
            connections.append(connection_dict)
    
        # Prepare the data 
        new_dict = {'country_code':country['Code'], 'town':location['AddressInfo']['Town'], 'location_name':location['AddressInfo']['Title'], 'Latitude':location['AddressInfo']['Latitude'], 'Longitude':location['AddressInfo']['Longitude'], 'charging_points': location['NumberOfPoints'], 'user_comments': location['GeneralComments'], 'connections': connections} 
    
        # Insert a new record in the Charging Station Location collection
        db.charging_station.insert_one(new_dict)
    
    time.sleep(1) # Break between API calls

print("COMPLETE - ALL API data retrieved and stored in Mongo")

'Trying Afghanistan ....'
'Trying Åland Islands ....'
'Trying Albania ....'
'Trying Algeria ....'
'Trying American Samoa ....'
'Trying Andorra ....'
'Trying Angola ....'
'Trying Anguilla ....'
'Trying Antarctica ....'
'Trying Antigua and Barbuda ....'
'Trying Argentina ....'
'Trying Armenia ....'
'Trying Aruba ....'
'Trying Australia ....'
'Trying Austria ....'
'Trying Azerbaijan ....'
'Trying Bahamas ....'
'Trying Bahrain ....'
'Trying Bangladesh ....'
'Trying Barbados ....'
'Trying Belarus ....'
'Trying Belgium ....'
'Trying Belize ....'
'Trying Benin ....'
'Trying Bermuda ....'
'Trying Bhutan ....'
'Trying Bolivia, Plurinational State of ....'
'Trying Bonaire, Sint Eustatius and Saba ....'
'Trying Bosnia and Herzegovina ....'
'Trying Botswana ....'
'Trying Bouvet Island ....'
'Trying Brazil ....'
'Trying British Indian Ocean Territory ....'
'Trying Brunei Darussalam ....'
'Trying Bulgaria ....'
'Trying Burkina Faso ....'
'Trying Burundi ....'
'Trying Cambodia ....'
'Trying Cameroon 